# Configurações iniciais

In [1]:
# Import de bibliotecas
from zipfile import ZipFile  # Leitura de arquivos .zip
import requests  # Requisição web
from io import BytesIO  # Leitura de arquivo em bytes
import pandas as pd  # Manipulação de dados


pd.options.display.max_columns = None

2024-02-15 23:09:30,924 [logging.log_init] INFO: LOGLEVEL=INFO


# Extração dos dados

In [2]:
# Extrair o arquivo excel da pasta ZIP diretamente do site de download
url = 'https://www.b3.com.br/data/files/57/E6/AA/A1/68C7781064456178AC094EA8/ClassifSetorial.zip'
response = requests.get(url=url)
with ZipFile(BytesIO(response.content)) as fold:
    file = fold.namelist()[0]
    with fold.open(file) as file:
        df = pd.read_excel(io=file, skiprows=6)

# Transformação dos dados

In [ ]:
df

In [3]:
df = df.rename(columns={'SEGMENTO': 'NOME', 'LISTAGEM': 'CÓDIGO', 'Unnamed: 4': 'LISTAGEM'})[1:]
df.loc[(df['CÓDIGO'].isnull()), 'SEGMENTO'] = df.loc[(df['CÓDIGO'].isnull()), 'NOME']
df = df.dropna(how='all')
df = df[['SETOR ECONÔMICO', 'SUBSETOR', 'SEGMENTO', 'NOME', 'CÓDIGO', 'LISTAGEM']]
df['SETOR ECONÔMICO'].ffill(inplace=True)
df['SETOR ECONÔMICO'].astype('category')
df['SUBSETOR'].ffill(inplace=True)
df['SUBSETOR'].astype('category')
df['SEGMENTO'].ffill(inplace=True)
df['SEGMENTO'].astype('category')
df = df.loc[(df.CÓDIGO!='CÓDIGO') & (df.CÓDIGO!='LISTAGEM') & (~df.CÓDIGO.isnull())]
display(df)
df.to_parquet(path='./tickers_por_setor.parquet')

In [2]:
all_papes = fundamentus.list_papel_all()

df = fundamentus.get_papel('BBAS3')

c:\Users\adria\anaconda3\Lib\site-packages\fundamentus\detalhes.py:264: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(content.text, decimal=",", thousands='.')[0]
2024-02-15 22:19:23,615 [detalhes.list_papel_all] INFO: members in list = 1032
2024-02-15 22:19:23,616 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
c:\Users\adria\anaconda3\Lib\site-packages\fundamentus\detalhes.py:232: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables_html = pd.read_html(content.text, decimal=",", thousands='.')


In [14]:
df.columns

Index(['SETOR ECONÔMICO', 'SUBSETOR', 'NOME', 'CÓDIGO', 'LISTAGEM',
       'SEGMENTO'],
      dtype='object')